In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [ ]:
from ipynb.fs.full.LIGO_data_simulation_functions import noise, wave, dataset, norm

In [ ]:
fs = 4096
dt = 1/fs
data_size = 5*fs
sample_size = 10000

In [ ]:
train_data, train_labels= dataset(sample_size,data_size)
test_data, test_labels = dataset(sample_size,data_size)
train_data = norm(train_data)
test_data = norm(test_data)

In [ ]:
train_chirp_m = train_labels[:,0]
train_d = train_labels[:,1]
test_chirp_m = test_labels[:,0]
test_d = test_labels[:,1]

In [ ]:
train_pm = train_labels[:,:2]
test_pm = test_labels[:,:2]

In [ ]:
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/train data.npz", train_data = train_data)
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/train labels.npz", train_labels = train_labels)
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/test data.npz", test_data = test_data)
np.savez("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/test labels.npz", test_labels = test_labels)

In [ ]:
train_data = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/train data.npz")['train_data']
train_labels = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/train labels.npz")['train_labels']
test_data = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/test data.npz")['test_data']
test_labels = np.load("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/test labels.npz")['test_labels']

In [ ]:
model = models.Sequential([
    layers.Reshape((train_data.shape[-1],1),input_shape=[train_data.shape[-1]]),
    layers.Conv1D(16,3, activation='relu'),
    layers.MaxPooling1D(4),
    layers.Dense(16, activation='relu'),
    layers.Conv1D(8,3, activation='relu'),
    layers.MaxPooling1D(4),
    layers.Dense(8, activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(2, activation='linear')
  ])
model.compile(optimizer='adam',
              loss='mean_squared_error', 
              metrics=['mean_squared_error'])
model.summary()

In [ ]:
history = model.fit(train_data, train_pm, epochs=30, #30 epochs
                    validation_data=(test_data, test_pm))

In [ ]:
model.save_weights("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/updated_pm_model_weights.ckpt".format(epoch=30))

In [ ]:
model.load_weights("/Users/jkliao117/Desktop/MSci/updated simulated data/pm/updated_pm_model_weights.ckpt")

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_pm, verbose=2)

In [ ]:
predictions = np.reshape(model.predict(test_data),(sample_size,2))
difference = test_pm-predictions

In [ ]:
plt.figure(figsize=(8,6))
x = np.arange(0,100,1)
plt.scatter(test_pm[:,0],predictions[:,0],alpha=0.25)
plt.plot(x,x,'r',alpha=0.5)
plt.xlim(0,100)
plt.ylim(0,100)
plt.grid()
plt.xlabel("True Chirp Mass ($M_{\odot}$)",fontsize=20)
plt.ylabel("Predicted Chirp Mass ($M_{\odot}$)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.tight_layout()
#plt.savefig('/Users/jkliao117/Desktop/prediction vs label', transparent=True)

In [ ]:
chirp_m_range = np.arange(10,100,10)
avg_chirp_m_difference = np.zeros(len(chirp_m_range)-1)
avg_chirp_m_pd = np.zeros(len(chirp_m_range)-1)
avg_chirp_m_mean = np.zeros(len(chirp_m_range)-1)
avg_chirp_m_std = np.zeros(len(chirp_m_range)-1)
for i in range(len(chirp_m_range)-1):
    mask = [(test_chirp_m>chirp_m_range[i]-5)&(test_chirp_m<chirp_m_range[i]+5)]
    avg_chirp_m_difference[i] = np.mean(np.abs(difference)[:,0][mask])
    avg_chirp_m_pd[i] = np.mean(np.abs(difference)[:,0][mask]/test_chirp_m[mask])
    avg_chirp_m_mean[i] = np.mean(predictions[:,0][mask])
    avg_chirp_m_std[i] = np.std(np.abs(difference)[:,0][mask])

In [ ]:
plt.figure(figsize=(8,6))
x = np.arange(0,100,1)
plt.errorbar(chirp_m_range[:-1],avg_chirp_m_mean,yerr=avg_chirp_m_std,fmt='o',capsize=5)
plt.plot(x,x,'r',alpha=0.5)
plt.xlim(0,100)
plt.ylim(0,100)
plt.grid()
plt.xlabel("True Chirp Mass ($M_{\odot}$)",fontsize=20)
plt.ylabel("Predicted Chirp Mass ($M_{\odot}$)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.tight_layout()
#plt.savefig('/Users/jkliao117/Desktop/prediction vs label', transparent=True)

In [ ]:
plt.figure(figsize=(8,6))
x = np.arange(0,1100,1)
plt.scatter(test_pm[:,1],predictions[:,1],alpha=0.25)
plt.plot(x,x,'r',alpha=0.5)
plt.xlim(10,1100)
plt.ylim(10,1100)
plt.grid()
plt.xlabel("True Luminosity Distance ($Mpc$)",fontsize=20)
plt.ylabel("Predicted Luminosity Distance ($Mpc$)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.tight_layout()
#plt.savefig('/Users/jkliao117/Desktop/prediction vs label', transparent=True)

In [ ]:
d_range = np.arange(100,1200,100)
avg_d_difference = np.zeros(len(d_range)-1)
avg_d_pd = np.zeros(len(d_range)-1)
avg_d_mean = np.zeros(len(d_range)-1)
avg_d_std = np.zeros(len(d_range)-1)
for i in range(len(d_range)-1):
    mask = [(test_d>d_range[i]-50)&(test_d<d_range[i]+50)]
    avg_d_difference[i] = np.mean(np.abs(difference)[:,1][mask])
    avg_d_pd[i] = np.mean(np.abs(difference)[:,1][mask]/test_chirp_m[mask])
    avg_d_mean[i] = np.mean(predictions[:,1][mask])
    avg_d_std[i] = np.std(np.abs(difference)[:,1][mask])

In [ ]:
plt.figure(figsize=(8,6))
x = np.arange(0,1100,1)
plt.errorbar(d_range[:-1],avg_d_mean,yerr=avg_d_std,fmt='o',capsize=5)
plt.plot(x,x,'r',alpha=0.5)
plt.xlim(10,1100)
plt.ylim(10,1100)
plt.grid()
plt.xlabel("True Luminosity Distance ($Mpc$)",fontsize=20)
plt.ylabel("Predicted Luminosity Distance ($Mpc$)",fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.tight_layout()
#plt.savefig('/Users/jkliao117/Desktop/prediction vs label', transparent=True)

In [ ]:
chirp_m_pe = np.mean(np.abs(difference[:,0])/test_chirp_m)
print(chirp_m_pe)

In [ ]:
d_pe = np.mean(np.abs(difference[:,1])/test_d)
print(d_pe)

In [ ]:
# real LIGO events chirp mass
real_chirp_m = np.array([28.6,15.2,8.9,21.5,7.9,35.7,25,24.2,1.186,26.7,29.3])
real_chirp_m_error = np.array([1.5,1.1,0.3,1.7,0.2,4.7,1.6,1.1,0.001,1.7,3.2])

In [ ]:
# real LIGO events distance 
real_d = np.array([430,1060,440,960,320,2750,990,580,40,1020,1850])
real_d_error = np.array([150,480,180,410,110,1320,320,160,10,360,840])

In [ ]:
real_chirp_m_pe = np.mean(real_chirp_m_error/real_chirp_m)
print(real_chirp_m_pe)

In [ ]:
real_d_pe = np.mean(real_d_error/real_d)
print(real_d_pe)